In [15]:
import QuantLib as ql
import numpy as np
import scipy.stats as stats
import scipy.optimize as spo
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from random import sample
import sklearn.neural_network as sknn
import matplotlib.pyplot as plt
import time
from scipy.integrate import *
import cmath as cpx
import math as m
from sklearn.metrics import mean_squared_error
from copy import deepcopy

In [2]:
def MakeUniformRandParams(ArrayMin,ArrayMax,nRand):
    if (len(ArrayMin)!=len(ArrayMax)):
        raise Exception('ArrayMax and ArrayMin must have the same size')
    else:
        nParams = len(ArrayMin)
        rand_seq = ql.UniformRandomSequenceGenerator(nParams*nRand,ql.UniformRandomGenerator())
        X = rand_seq.nextSequence().value()
        #X = [ArrayMin[j%nParams]+(ArrayMax[j%nParams]-ArrayMin[j%nParams])*X[j] for j in range(nParams*nRand)]
        res = np.zeros((nRand,nParams))
        LocalMax = 0
        res__ = 0
        res_ = 0
        for i in range(nRand):
            for j in range(nParams):
                AdjMax = ArrayMax[j]
                if j==2:
                    AdjMax = XiFellerAdjust(res__,res_,ArrayMax[j])
                res[i,j]=ArrayMin[j]+(AdjMax-ArrayMin[j])*X[i*nParams+j]
                res__ = res_
                res_ = res[i,j]
        #res = np.reshape(X,(nRand,nParams))
        return res

def CountNoFellerCondition(X):
    k=0
    for j in range(len(X)):
        if (X[j,2]>XiFellerMax(X[j,0],X[j,1])):
            k=k+1
    return k

def XiFellerMax(kappa,theta):
    return m.sqrt(2*kappa*theta)

def XiFellerAdjust(kappa,theta,xi_init):
    if (xi_init>XiFellerMax(kappa,theta)):
        return XiFellerMax(kappa,theta)
    return xi_init
    
ArrayMin = [0.1, 0.1,  0.01, -0.999, 0.1]
ArrayMax = [1.5, 1.5,  0.75,  0.999, 1.5]
nRand = 100000
start = time.time()
res=MakeUniformRandParams(ArrayMin,ArrayMax,nRand)
end = time.time()
print('Total computation time = ' + str(round(end-start,2)) + 's')
print('No Feller Condition ' + str(CountNoFellerCondition(res)))

Total computation time = 0.98s
No Feller Condition 0


In [3]:
# GLOBAL CONFIG #
AsOfDate = ql.Date(15,10,2018)
ql.Settings.instance().evaluationDate = AsOfDate
DayCount = ql.Actual365Fixed()
Calendar = ql.UnitedStates()

In [61]:
# EUROPEAN OPTION CONFIG #
MaturityDates = []
strikes = []
exercises = []
payoffs = []
option_type = ql.Option.Call
ms = [1,2,3,6,0,6,0,0,0,0]
ys = [0,0,0,0,1,1,2,3,4,5]
for k in range(len(ms)):
    if (10+ms[k]>12):
        ys_sup=1
    else:
        ys_sup=0
    MaturityDates.append(ql.Date(15,(10+ms[k]-1)%12+1,2018+ys[k]+ys_sup))
    exercises.append(ql.EuropeanExercise(MaturityDates[k]))
for k in range(7):
        strikes.append(70+k*10)
        #payoffs.append(ql.PlainVanillaPayoff(option_type, strikes[k]))
print(MaturityDates)
print(strikes)
EuropeanOptions = []
for T in range(len(MaturityDates)):
    for K in range(len(strikes)):
        EuropeanOptions.append(ql.VanillaOption(ql.PlainVanillaPayoff(option_type,100-(100-strikes[K])*m.sqrt((MaturityDates[T]-AsOfDate)/365)), exercises[T]))
#print(EuropeanOptions)

[Date(15,11,2018), Date(15,12,2018), Date(15,1,2019), Date(15,4,2019), Date(15,10,2019), Date(15,4,2020), Date(15,10,2020), Date(15,10,2021), Date(15,10,2022), Date(15,10,2023)]
[70, 80, 90, 100, 110, 120, 130]


In [137]:
def d1(S0, K, r, sigma, T):
    m = S0/(K*np.exp(-r*T))
    return np.log(m)*(1/(sigma*np.sqrt(T)))+sigma*np.sqrt(T)/2

def d2(S0, K, r, sigma, T):
    return d1(S0,K,r,sigma,T)-sigma*np.sqrt(T)
 
def BS_CallPrice(S0,K,r,sigma,T):
    if T==0:
        return np.maximum(S0-K,0)
    else:
        return S0*stats.norm.cdf(d1(S0,K,r,sigma,T))-K*np.exp(-r*T)*stats.norm.cdf(d2(S0,K,r,sigma,T))

def Heston_CharacteristicFunctionLogPrice(t,omega,S0,v0,r,kappa,theta,ksi,rho):
    alpha = complex(-omega**2 / 2, -omega / 2)
    beta = complex(kappa,-rho*ksi*omega)
    gamma = ksi**2 / 2
    h = (beta**2-4*alpha*gamma)**0.5
    r_m = (beta-h) / ksi**2
    r_p = (beta+h) / ksi**2
    g = r_m / r_p
    C = kappa*(r_m*t-2*cpx.log((1-g*cpx.exp(-h*t))/(1-g))/ksi**2)
    D = r_m*((1-cpx.exp(-h*t))/(1-g*cpx.exp(-h*t)))
    return cpx.exp(complex(C*theta+D*v0,omega*np.log(S0*np.exp(r*t))))

def RPI1_HestonChFuncLnS(omega,K,t,S0,v0,r,kappa,theta,ksi,rho):
    num = Heston_CharacteristicFunctionLogPrice(t,omega-complex(0,1),S0,v0,r,kappa,theta,ksi,rho)*cpx.exp(complex(0,-omega*np.log(K)))
    denom = complex(0,omega*Heston_CharacteristicFunctionLogPrice(t,-complex(0,1),S0,v0,r,kappa,theta,ksi,rho))
    return (num / denom).real
def RPI2_HestonChFuncLnS(omega,K,t,S0,v0,r,kappa,theta,ksi,rho):
    num = Heston_CharacteristicFunctionLogPrice(t,omega,S0,v0,r,kappa,theta,ksi,rho)*cpx.exp(complex(0,-omega*np.log(K)))
    denom = complex(0,omega)
    return (num / denom).real
def Heston_CallPrice(K,S0,r,T,v0,kappa,theta,ksi,rho):
    Pi_1 = 1/2+1/np.pi*quad(RPI1_HestonChFuncLnS,0,np.inf,args=(K,T,S0,v0,r,kappa,theta,ksi,rho))[0]
    Pi_2 = 1/2+1/np.pi*quad(RPI2_HestonChFuncLnS,0,np.inf,args=(K,T,S0,v0,r,kappa,theta,ksi,rho))[0]
    return S0*Pi_1-K*np.exp(-r*T)*Pi_2

def IVol(Mkt,S0,K,r,T,vol0=0.1):
    def f(vol,Mkt_,S0_,K_,r_,T_,type_): return (BS_CallPrice(S0_,K_,r_,vol,T_)-Mkt)**2
    return spo.minimize(f,vol0,args=(Mkt,S0,K,r,T,type),tol=1.0e-10).x

In [5]:
# EUROPEAN OPTION CONFIG V2 - necessary in V1 for numeric values #
Expiries = [0.0833,0.1667,0.25,0.5,1,1.5,2,3]
BaseStrikes = [70,80,90,100,110,120,130]
Strikes = []
for T in range(len(Expiries)):
    for K in range(len(BaseStrikes)):
        Strikes.append(100-(100-BaseStrikes[K])*m.sqrt(Expiries[T])) 

In [62]:
# MARKET DATA
r=ql.SimpleQuote(0.01)
r_ts = ql.FlatForward(0,Calendar,ql.QuoteHandle(r),DayCount)
q=ql.SimpleQuote(0.0)
q_ts = ql.FlatForward(0,Calendar,ql.QuoteHandle(q),DayCount)
s0=ql.SimpleQuote(100)

# BLACK SCHOLES ENGINE ONLY USED FOR COMPUTING IMPLIED VOLATILITIES
vol_bs = ql.SimpleQuote(0.10)
vol_bs_ts = ql.BlackConstantVol(0,Calendar,ql.QuoteHandle(vol_bs),DayCount)
BSProcess = ql.BlackScholesMertonProcess(ql.QuoteHandle(s0),ql.YieldTermStructureHandle(r_ts),ql.YieldTermStructureHandle(q_ts),
                               ql.BlackVolTermStructureHandle(vol_bs_ts))

# HESTON ENGINE
v0 = 0.01
kappa=0.5
theta=0.01
sigma=0.0001
rho=-0.5
HestonParams = [theta,kappa,sigma,rho,v0]
relTolerance=0.01
maxEval=10000
HestonProcess = ql.HestonProcess(ql.YieldTermStructureHandle(r_ts),ql.YieldTermStructureHandle(q_ts),
                                 ql.QuoteHandle(s0),v0,kappa,theta,sigma,rho)
HestonModel = ql.HestonModel(HestonProcess)
HestonEngine = ql.AnalyticHestonEngine(HestonModel,relTolerance,maxEval)
for i in range(len(EuropeanOptions)):
    EuropeanOptions[i].setPricingEngine(HestonEngine)

In [66]:
# GENERATE DATAS
nRand = 50000
start = time.time()
MinParamsValues = [0.1,0.1,0.1,-0.999,0.1]
MaxParamsValues = [2.5,0.6,1.5,-0.10,0.6]
local_start = 0
local_end = 0
ArrParams = MakeUniformRandParams(MinParamsValues,MaxParamsValues,nRand)
intermediary = time.time()

#npv = np.zeros(nRand)
#iv = np.zeros(nRand)
#for i in range(nRand):
#    HestonModel.setParams([ArrParams[i][k] for k in range(len(MinParamsValues))])
#    npv[i] = EuropeanOption.NPV()
#    iv[i] = EuropeanOption.impliedVolatility(npv[i],BSProcess,1.0e-4,200,1.0e-8,10)
#end = time.time()
#print('Random params simulation time = ' + str(round(intermediary-start,2)) + 's')
#print('Prices computation time = ' + str(round(end-intermediary,2)) + 's')
#print('Total computation time = ' + str(round(end-start,2)) + 's')
#X = np.zeros((nRand*len(Strikes),len(ArrParams[0])+2))
npv = 0
iv = np.zeros((nRand,len(EuropeanOptions)))
for i in range(nRand):
    HestonModel.setParams([ArrParams[i,k] for k in range(len(MinParamsValues))])
    for j in range(len(EuropeanOptions)):
        npv = max(EuropeanOptions[j].NPV(),np.random.uniform(0,0.01))
        iv[i,j] = EuropeanOptions[j].impliedVolatility(npv,BSProcess,1.0e-4,200,1.0e-8,100)
end = time.time()
print('Random params simulation time = ' + str(round(intermediary-start,2)) + 's')
print('Prices computation time = ' + str(round(end-intermediary,2)) + 's')
print('Total computation time = ' + str(round(end-start,2)) + 's')

Random params simulation time = 0.43s
Prices computation time = 634.59s
Total computation time = 635.02s


In [148]:
# GENERATE DATAS V2
nRand = 10
start = time.time()
S0 = 100
r = 0.01
MinParamsValues = [0.1,0.1,0.1,-0.999,0.1]
MaxParamsValues = [2.5,0.5,1.5,-0.10,0.5]
local_start = 0
local_end = 0
ArrParams = MakeUniformRandParams(MinParamsValues,MaxParamsValues,nRand)
intermediary = time.time()

#npv = np.zeros(nRand)
#iv = np.zeros(nRand)
#for i in range(nRand):
#    HestonModel.setParams([ArrParams[i][k] for k in range(len(MinParamsValues))])
#    npv[i] = EuropeanOption.NPV()
#    iv[i] = EuropeanOption.impliedVolatility(npv[i],BSProcess,1.0e-4,200,1.0e-8,10)
#end = time.time()
#print('Random params simulation time = ' + str(round(intermediary-start,2)) + 's')
#print('Prices computation time = ' + str(round(end-intermediary,2)) + 's')
#print('Total computation time = ' + str(round(end-start,2)) + 's')
npv = np.zeros(nRand*len(Strikes))
iv = np.zeros(nRand*len(Strikes))
for i in range(nRand):
    kappa = ArrParams[i,0]
    theta = ArrParams[i,1]
    xi = ArrParams[i,2]
    rho = ArrParams[i,3]
    v0 = ArrParams[i,4]
    j = 0
    for T in range(len(Expiries)):
        for K in range(len(BaseStrikes)):
            npv[i*len(Strikes)+j] = Heston_CallPrice(Strikes[j],S0,r,Expiries[T],v0,kappa,theta,xi,rho)
            iv[i*len(Strikes)+j] = IVol(npv[i*len(Strikes)+j],S0,Strikes[j],r,Expiries[T])
            j = j+1
end = time.time()
print('Random params simulation time = ' + str(round(intermediary-start,2)) + 's')
print('Prices computation time = ' + str(round(end-intermediary,2)) + 's')
print('Total computation time = ' + str(round(end-start,2)) + 's')

Random params simulation time = 0.0s
Prices computation time = 14.46s
Total computation time = 14.46s


In [58]:
HestonPrice = EuropeanOptions[j].NPV()
time_=ql.Date(15,11,2018)-AsOfDate
print(time_)
print(EuropeanOptions[j])

31
<QuantLib.QuantLib.VanillaOption; proxy of <Swig Object of type 'VanillaOptionPtr *' at 0x00000263EC97A2D0> >


In [60]:
print(HestonPrice)
print('K = ' + str(strikes[j%len(strikes)]))
print('T = ' + str(MaturityDates[int(j/len(strikes))]))
print(ArrParams[i])
new_vol = 100
vol_bs.setValue(new_vol)
EuropeanOptions[j].setPricingEngine(ql.AnalyticEuropeanEngine(BSProcess))
print(EuropeanOptions[j].NPV())

97.3865967715769
K = 70
T = October 15th, 2028
[ 0.87791359  0.57905022  0.85163063 -0.6712743   0.47793902]
90.47630508934425


In [9]:
def ann_layers_neurones(layer, neurone):
    hidden_layer_sizes = (neurone, )
    if layer == 1.0: return hidden_layer_sizes
    elif layer > 1.0:
        for i in range(layer-1):
            hidden_layer_sizes += (neurone,)
        return hidden_layer_sizes

nLayersMin = 3
nLayersMax = 5
nLayersStep = 1
nNeuronesMin = 30
nNeuronesMax = 60
nNeuronesStep = 5
hidden_layers_sizes_vect = []
i_ = 0
for i in range(nNeuronesMin,nNeuronesMax+1,nNeuronesStep):
    j_ = 0
    for j in range(nLayersMin,nLayersMax+1,nLayersStep):
        hidden_layers_sizes_vect.append(ann_layers_neurones(j,i))
        j_ += 1
    i_ += 1
print(hidden_layers_sizes_vect)

[(30, 30, 30), (30, 30, 30, 30), (30, 30, 30, 30, 30), (35, 35, 35), (35, 35, 35, 35), (35, 35, 35, 35, 35), (40, 40, 40), (40, 40, 40, 40), (40, 40, 40, 40, 40), (45, 45, 45), (45, 45, 45, 45), (45, 45, 45, 45, 45), (50, 50, 50), (50, 50, 50, 50), (50, 50, 50, 50, 50), (55, 55, 55), (55, 55, 55, 55), (55, 55, 55, 55, 55), (60, 60, 60), (60, 60, 60, 60), (60, 60, 60, 60, 60)]


In [67]:
start = time.time()
n = nRand
ann = sknn.MLPRegressor(hidden_layer_sizes=(30,30,30))
array_train_rel_error = np.zeros((n,len(EuropeanOptions)))
array_test_rel_error = np.zeros((n,len(EuropeanOptions)))
array_train_abs_error = np.zeros((n,len(EuropeanOptions)))
array_test_abs_error = np.zeros((n,len(EuropeanOptions)))
index = sample(range(0,nRand),n)
X=np.zeros((n,len(ArrParams[0])))
for k in range(len(ArrParams[0])):        
    X[:,k]=[ArrParams[j,k] for j in index]
Y=[iv[j,:] for j in index]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# ANN FIT

mse_optim = 1000
ann_optim = deepcopy(ann)
for current_archi in hidden_layers_sizes_vect:
    ann.set_params(hidden_layer_sizes=current_archi)
    ann.fit(X_train,Y_train)
    train_prediction=ann.predict(X_train)
    mse=mean_squared_error(Y_train,train_prediction)
    print('current archi ' + str(current_archi) + ' ; mse = ' + str(mse))
    if (mse<mse_optim):
        mse_optim=mse
        print('I am optimal')
        ann_optim=deepcopy(ann)
        test_prediction=ann.predict(X_test)
        for i in range(max(len(X_train),len(X_test))):
            for j in range(len(EuropeanOptions)):
                if (i < len(X_train)):
                    array_train_abs_error[i,j]=(train_prediction[i,j]-Y_train[i][j])*100
                    array_train_rel_error[i,j]=(train_prediction[i,j]/Y_train[i][j]-1)*100
                if (i < len(X_test)):
                    array_test_abs_error[i,j]=(test_prediction[i,j]-Y_test[i][j])*100
                    array_test_rel_error[i,j]=(test_prediction[i,j]/Y_test[i][j]-1)*100
        #array_train_error=[[ann.predict(X_train)[i,j]-Y_train[i][j] for j in range(len(EuropeanOptions))] for i in range(n)]
        #array_test_error=[[ann.predict(X_test)[i,j]-Y_test[i][j] for j in range(len(EuropeanOptions))] for i in range(n)]
end = time.time()
print('Total computation time = ' + str(round(end-start,2)) + 's')

current archi (30, 30, 30) ; mse = 0.0002326441090707351
I am optimal
current archi (30, 30, 30, 30) ; mse = 0.00025027419191350856
current archi (30, 30, 30, 30, 30) ; mse = 0.00023940090084760462
current archi (35, 35, 35) ; mse = 0.0002357998408421149
current archi (35, 35, 35, 35) ; mse = 0.00019005807882527538
I am optimal
current archi (35, 35, 35, 35, 35) ; mse = 0.00019473634666146824
current archi (40, 40, 40) ; mse = 0.00027821166775488605
current archi (40, 40, 40, 40) ; mse = 0.0001928114910140233
current archi (40, 40, 40, 40, 40) ; mse = 0.00012824412391278757
I am optimal
current archi (45, 45, 45) ; mse = 0.00018288268637686272
current archi (45, 45, 45, 45) ; mse = 0.00015184155195019307
current archi (45, 45, 45, 45, 45) ; mse = 8.637649741203321e-05
I am optimal
current archi (50, 50, 50) ; mse = 0.00016224134102630445
current archi (50, 50, 50, 50) ; mse = 0.0001910399466530497
current archi (50, 50, 50, 50, 50) ; mse = 0.00011192871605907136
current archi (55, 55, 

In [32]:
print(ann_optim.get_params())

{'activation': 'relu', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (60, 60, 60, 60, 60), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_iter': 200, 'momentum': 0.9, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}


In [47]:
mean_abs_train = np.mean(array_train_abs_error,axis=0)
mean_abs_test = np.mean(array_test_abs_error,axis=0)
std_abs_train = np.std(array_train_abs_error,axis=0)
std_abs_test = np.std(array_test_abs_error,axis=0)
max_abs_train = np.amax(np.absolute(array_train_abs_error),axis=0)
max_abs_test = np.amax(np.absolute(array_test_abs_error),axis=0)

mean_rel_train = np.mean(array_train_rel_error,axis=0)
mean_rel_test = np.mean(array_test_rel_error,axis=0)
std_rel_train = np.std(array_train_rel_error,axis=0)
std_rel_test = np.std(array_test_rel_error,axis=0)
max_rel_train = np.amax(np.absolute(array_train_rel_error),axis=0)
max_rel_test = np.amax(np.absolute(array_test_rel_error),axis=0)

mean_abs_abs_train=np.mean(np.absolute(array_train_abs_error),axis=0)
mean_abs_rel_train=np.mean(np.absolute(array_train_rel_error),axis=0)
mean_abs_abs_test=np.mean(np.absolute(array_test_abs_error),axis=0)
mean_abs_rel_test=np.mean(np.absolute(array_test_rel_error),axis=0)

print("mean_abs_abs_train = ")
print(mean_abs_abs_train)
print('\n')
print("mean_abs_rel_train = ")
print(mean_abs_rel_train)
print('\n')
print("mean_abs_abs_test = ")
print(mean_abs_abs_test)
print('\n')
print("mean_abs_rel_test = ")
print(mean_abs_rel_test)
print('\n')
print("mean_abs_train = ")
print(mean_abs_train)
print('\n')
print("mean_abs_test = ")
print(mean_abs_test)
print('\n')
print("std_abs_train = ")
print(std_abs_train)
print('\n')
print("std_abs_test = ")
print(std_abs_test)
print('\n')
print("max_abs_train = ")
print(max_abs_train)
print('\n')
print("max_abs_test = ")
print(max_abs_test)
print('\n')
print("mean_rel_train = ")
print(mean_rel_train)
print('\n')
print("mean_rel_test = ")
print(mean_rel_test)
print('\n')
print("std_rel_train = ")
print(std_rel_train)
print('\n')
print("std_rel_test = ")
print(std_rel_test)
print('\n')
print("max_rel_train = ")
print(max_rel_train)
print('\n')
print("max_rel_test = ")
print(max_rel_test)
print('\n')

mean_abs_abs_train = 
[0.42694751 0.51739155 0.44251591 0.43597408 0.4835116  0.53362715
 0.59838676 0.44061864 0.46049519 0.49052012 0.44574046 0.48456501
 0.52758303 0.53900834 0.42901162 0.39606571 0.47353473 0.40416138
 0.42892496 0.43450535 0.45895857 0.44503915 0.44811736 0.53145475
 0.42352073 0.49012365 0.46030059 0.49009124 0.52250347 0.43236853
 0.43810921 0.4691864  0.53581207 0.50023819 0.49578114 0.55870597
 0.48937535 0.50097095 0.52777337 0.56466276 0.49698092 0.55198211
 0.55976229 0.48636477 0.56380453 0.47634567 0.52149403 0.55851986
 0.54882303 0.68490206 0.68310864 0.64884111 0.64038548 0.64579617
 0.73252792 0.63268088]


mean_abs_rel_train = 
[0.74667418 0.91943405 0.80744593 0.78715782 0.88939182 0.99119478
 1.08786288 0.74555131 0.77722962 0.85063325 0.77929842 0.85932673
 0.95112661 0.9793246  0.69978258 0.65205532 0.80595172 0.6857006
 0.75084288 0.76739609 0.80183439 0.6743287  0.70133486 0.82553323
 0.68581619 0.79565945 0.7777592  0.83883595 0.73324782 0.62

In [71]:
def EvalANN(params,ann_,output_): return mean_squared_error(output_,ann_.predict([params]))
def InvertANN(ann,mkt,scaler,params0=[0.5,0.25,0.3,-0.5,0.25]):
    Sparams0 = scaler.transform([params0])
    return spo.minimize(EvalANN,Sparams0,args=(ann,mkt),tol=1.0e-8)

nRand = 1
RefParams = MakeUniformRandParams(MinParamsValues,MaxParamsValues,nRand)

npv = 0
iv = np.zeros((nRand,len(EuropeanOptions)))
for i in range(nRand):
    HestonModel.setParams([RefParams[i,k] for k in range(len(MinParamsValues))])
    for j in range(len(EuropeanOptions)):
        npv = max(EuropeanOptions[j].NPV(),np.random.uniform(0,0.01))
        iv[i,j] = EuropeanOptions[j].impliedVolatility(npv,BSProcess,1.0e-4,200,1.0e-8,100)

start = time.time()
res = InvertANN(ann_optim,iv,scaler)
end = time.time()
ann_iv = ann_optim.predict([res.x])
print('Calibration computation time = ' + str(round(end-start,2)) + 's')
print('The optimal set of params is ' + str(scaler.inverse_transform(res.x)))
print('The initial set of params is ' + str(RefParams[0]))
print('The standard deviation on the surface is ' + str(m.sqrt(res.fun)*100) + '%')
print('\n')
print('Initial IV Surface : ' + str(iv[0]*100))
print('\n')
print('ANN Calibrated IV Surface : ' + str(ann_iv[0]*100))
print('\n')

print('Error ABS IV Surface : ' + str((iv-ann_iv)[0]*100))
#print('The optimal set of params is ' + str(res.x))
#EvalANN([0.5,0.25,0.3,-0.5,0.25],ann_optim,iv)

Calibration computation time = 0.41s
The optimal set of params is [ 1.23900276  0.36494159  0.34046391 -0.51619445  0.48415174]
The initial set of params is [ 1.12235184  0.45829119  0.27471461 -0.62960429  0.48113012]
The standard deviation on the surface is 0.342851476506711%


Initial IV Surface : [ 71.73166497  71.39639006  71.09675029  70.82619954  70.57584038
  70.34518131  70.12897815  73.05742689  72.59930691  72.18001604
  71.80454135  71.46399263  71.15133551  70.8618651   74.28024963
  73.68731694  73.17570668  72.71958907  72.31171931  71.94034192
  71.59889184  77.26609478  76.41673327  75.70335223  75.08776675
  74.54829093  74.06587646  73.63109969  82.18093141  80.91838315
  79.91928392  79.09626915  78.39821322  77.79677462  77.2606886
  86.28230742  84.62993262  83.38877958  82.40998516  81.60533473
  80.92750577  80.34010781  89.93472429  87.84699368  86.37055924
  85.25059133  84.35558377  83.61319217  82.98412898  96.67484584
  93.43193394  91.40894381  89.98451513